# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [16]:
	
from pyspark.sql import SparkSession
from pyspark.sql.types import DateType
from pyspark.sql.functions import to_date
from pyspark.sql.functions import max
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [11]:
#write to parquet
try :
    df_spark.write.parquet("sas_data")
except :
    pass
df_spark=spark.read.parquet("sas_data")

In [12]:
df_spark.count()

3096313

In [13]:
df_spark.show(5)

+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|    cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|5748517.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     CA|20582.0|  40.0|    1.0|  1.0|20160430|     SYD| null|      G|      O|   null|      M| 1976.0|10292016|     F|  null|     QF|9.495387003E10|00011|      B1|
|5748518.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     NV|20591.0|  32.0|    1.0|  

In [17]:
df_spark.describe("i94yr").show()

+-------+--------------------+
|summary|               i94yr|
+-------+--------------------+
|  count|             3096313|
|   mean|              2016.0|
| stddev|4.282829613261096...|
|    min|              2016.0|
|    max|              2016.0|
+-------+--------------------+



In [18]:
df_spark.describe("i94mon").show()

+-------+-------+
|summary| i94mon|
+-------+-------+
|  count|3096313|
|   mean|    4.0|
| stddev|    0.0|
|    min|    4.0|
|    max|    4.0|
+-------+-------+



## Step 2: Explore and Assess the Data

### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### I94 Immigration Data

- I'll use the provided dataset sample in csv format to explore I94 Immigration data, and make some assumtions before working with the full dataset in Parquet format.
- Using the dataset sample I'll determine the needed columns, and the required cleansing steps, then apply that on the full parquet dataset.

In [17]:
pd.set_option('display.max_columns', 500)
img_df = pd.read_csv("immigration_data_sample.csv")

In [18]:
img_df.columns

Index(['Unnamed: 0', 'cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port',
       'arrdate', 'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa',
       'count', 'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd',
       'entdepu', 'matflag', 'biryear', 'dtaddto', 'gender', 'insnum',
       'airline', 'admnum', 'fltno', 'visatype'],
      dtype='object')

In [19]:
img_df.head(10)

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,NaN,NaN,G,O,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,NaN,G,R,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,2.0,1.0,20160407,NaN,NaN,G,O,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,2.0,1.0,20160428,DOH,NaN,G,O,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,2.0,1.0,20160406,NaN,NaN,Z,K,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT
5,721257,1481650.0,2016.0,4.0,577.0,577.0,ATL,20552.0,1.0,GA,20606.0,51.0,2.0,1.0,20160408,NaN,NaN,T,N,NaN,M,1965.0,10072016,M,NaN,DL,7.368526e+08,910,B2
6,1072780,2197173.0,2016.0,4.0,245.0,245.0,SFR,20556.0,1.0,CA,20635.0,48.0,2.0,1.0,20160412,NaN,NaN,T,O,NaN,M,1968.0,10112016,F,NaN,CX,7.863122e+08,870,B2
7,112205,232708.0,2016.0,4.0,113.0,135.0,NYC,20546.0,1.0,NY,20554.0,33.0,2.0,1.0,20160402,NaN,NaN,G,O,NaN,M,1983.0,06302016,F,NaN,BA,5.547449e+10,00117,WT
8,2577162,5227851.0,2016.0,4.0,131.0,131.0,CHI,20572.0,1.0,IL,20575.0,39.0,2.0,1.0,20160428,NaN,NaN,O,O,NaN,M,1977.0,07262016,NaN,NaN,LX,5.941342e+10,00008,WT
9,10930,13213.0,2016.0,4.0,116.0,116.0,LOS,20545.0,1.0,CA,20553.0,35.0,2.0,1.0,20160401,NaN,NaN,O,O,NaN,M,1981.0,06292016,NaN,NaN,AA,5.544979e+10,00109,WT


In [20]:
valid_cloumns = [ 'cicid', 'i94yr', 'i94mon', 'i94res', 'i94port', 'arrdate', 'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa', 'gender', 'airline', 'visatype']
invalid_columns = ['Unnamed: 0','i94cit', 'arrdate','count',  'dtadfile', 'visapost', 'occup','entdepa', 'entdepd','entdepu','matflag', 'biryear','dtaddto', 'insnum','admnum', 'fltno' ]

In [21]:
img_df = img_df[['cicid', 'i94yr', 'i94mon', 'i94res', 'i94port', 'arrdate', 'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa', 'gender', 'airline', 'visatype']]

In [22]:
img_df.head(10)

,cicid,i94yr,i94mon,i94res,i94port,i94mode,i94addr,depdate,i94bir,i94visa,gender,airline,visatype
0,4084316.0,2016.0,4.0,209.0,HHW,1.0,HI,20573.0,61.0,2.0,F,JL,WT
1,4422636.0,2016.0,4.0,582.0,MCA,1.0,TX,20568.0,26.0,2.0,M,*GA,B2
2,1195600.0,2016.0,4.0,112.0,OGG,1.0,FL,20571.0,76.0,2.0,M,LH,WT
3,5291768.0,2016.0,4.0,297.0,LOS,1.0,CA,20581.0,25.0,2.0,M,QR,B2
4,985523.0,2016.0,4.0,111.0,CHM,3.0,NY,20553.0,19.0,2.0,F,NaN,WT
5,1481650.0,2016.0,4.0,577.0,ATL,1.0,GA,20606.0,51.0,2.0,M,DL,B2
6,2197173.0,2016.0,4.0,245.0,SFR,1.0,CA,20635.0,48.0,2.0,F,CX,B2
7,232708.0,2016.0,4.0,135.0,NYC,1.0,NY,20554.0,33.0,2.0,F,BA,WT
8,5227851.0,2016.0,4.0,131.0,CHI,1.0,IL,20575.0,39.0,2.0,NaN,LX,WT
9,13213.0,2016.0,4.0,116.0,LOS,1.0,CA,20553.0,35.0,2.0,NaN,AA,WT


#### World Temperature Data

In [3]:
tmp_df = pd.read_csv('GlobalLandTemperaturesByState.csv')
#"../../data2/GlobalLandTemperaturesByCity.csv"

In [4]:
tmp_df.columns

Index(['dt', 'AverageTemperature', 'AverageTemperatureUncertainty', 'State',
       'Country'],
      dtype='object')

In [5]:
tmp_df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,State,Country
0,1855-05-01,25.544,1.171,Acre,Brazil
1,1855-06-01,24.228,1.103,Acre,Brazil
2,1855-07-01,24.371,1.044,Acre,Brazil
3,1855-08-01,25.427,1.073,Acre,Brazil
4,1855-09-01,25.675,1.014,Acre,Brazil


In [34]:
tmp_df.max()

dt                               2013-09-01
AverageTemperature                   39.651
AverageTemperatureUncertainty        15.396
City                                 Ürümqi
Country                            Zimbabwe
Latitude                              8.84S
Longitude                            99.91E
dtype: object

### Cleaning Steps
Document steps necessary to clean the data

#### I94 Immigration Data

*******1. I'll group data by countrt where imigrant comes from, and US state where immgirants enteres US
2. Remove all records where country (xxx) is Null
3. After understaning the dataset, I came up with list of columns that will server my project purpose as definded abov.
4. All undesired columns will be removed.
5. Drop all null and NaN
6. All numbers will be converted to Integers to maintain clean and consistant data type.
7. (i94addr) has a lot of values not correct US state abbrivaition. Will be changed to "99"
5. To avaoid unnecessary join, the codes in column (I94MODE) will be replaced with the values.
6. To avaoid unnecessary join, the codes in column (I94VISA) will be replaced with the values.
8. SAS date format (arrdate, depdate) will be converted to date. (https://communities.sas.com/t5/Developers/How-can-I-read-a-SAS-datetime-value-in-Python/td-p/433593)
11. Calculate stay length from arrival and departure dates.

#### World Temperature Data

1. I'll filter dataset to include only US data
2. I'll filter dataset to include only 2013 data or above.

#### U.S. City Demographic Data

#### Airport Code Table

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

cones